In [2]:
import pygame as pg
import pygame.draw as dw
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import sympy as smp

pygame 2.6.0 (SDL 2.28.4, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


# SYMPY
O código abaixo resolve analiticamente as eqs de Euler-Lagrange.

No final temos as equaçõs que descrevem o sistema. Se considerarmos $l$ constante, ficamos com as equações do costume. O que obtemos ao resolver em função de $\dot l,\ddot l, \dot\theta,\ddot\theta$ é aquilo que colocamos no RK4.

In [68]:
x, y, t, m, g = smp.symbols('x y t m g') # t é tempo
th, l = smp.symbols('theta l', cls=smp.Function)
th = th(t)
dth = smp.diff(th, t)
dth2 = smp.diff(dth, t)
l = l(t)
dl = smp.diff(l, t)
dl2 = smp.diff(dl, t)


x = l*smp.sin(th)
y = l*smp.cos(th)
dx = smp.diff(x,t)
dy = smp.diff(y,t)

T = 0.5*m*(dx**2 + dy**2)
U = m*g*y
L = T - U

# eq angulo
EL1 = smp.diff(L,th) - smp.diff(smp.diff(L,dth),t)
EL2 = smp.diff(L,l) - smp.diff(smp.diff(L,dl),t)

# resolver
sols = smp.solve([EL1,EL2], (dl2, dth2))
sols[dl2].simplify()
sols[dth2].simplify()


(1.0*g*sin(theta(t)) - 2.0*Derivative(l(t), t)*Derivative(theta(t), t))/l(t)

# RK4
Do tipo
$$\frac{dr}{dt} = f(x, t)$$

Para uma eq de movimento podemos ter $\frac{d^2\vec r}{dt^2}=f(\vec r,\frac{d\vec r}{dt}, t)$, que podemos resolver como:
$$\begin{cases}
\frac{d\vec r}{dt}=\vec v\\
\frac{d\vec v}{dt}= f(\vec r,\vec v,t)
\end{cases}$$

In [ ]:
# Aqui, a forma como se coloca r na função f() depende completamente de como f está definida e funciona

def RK4_2var(f, xi, yi, a, b, N):
    x, y = np.zeros(N), np.zeros(N)
    x[0] = xi
    y[0] = yi
    r = np.array([x,y]).T # transposta para cada linha ser um ponto (x,y)
    h = (b-a)/N
    t = np.linspace(a,b,N)
    for i in range(1, N):
        k1 = h * f(*r[i-1], t[i-1])
        k2 = h * f(*r[i-1] + 0.5*k1, t[i-1] + 0.5*h)
        k3 = h * f(*r[i-1] + 0.5*k2, t[i-1] + 0.5*h)
        k4 = h * f(*r[i-1] + k3, t[i-1] + h)

        r[i] = r[i-1] + 1/6*(k1 + 2*k2 + 2*k3 + k4)
    
    return (r.T), t

# Pêndulo Simples obtido com sympy ($\theta$ e $\ell$ variaveis)

In [82]:
comps = []

def animate():
    L = 1000
    disp = pg.display.set_mode((L,L), pg.RESIZABLE)
    pg.display.set_caption("Teste #1")
    clk = pg.time.Clock()
    FPS = 60

    # centro e raio de circulo
    C = np.array([L/2,L/2])
    dw.circle(disp, (255,255,255), C, 10, 0)

    # bola
    R = L/100
    r = C + np.array([L/15,0])
    comp = np.linalg.norm(r-C)
    th = np.arctan2((r-C)[0], (r-C)[1])
    # v = np.array([0,-1])
    # al = np.arctan2(v[0], v[1])
    # phi = np.pi/2 - al - th
    # om = np.linalg.norm(v) * np.cos(phi) / comp
    sol = np.array([th, 0, comp, 0])


    colorTraj = [100,100,100]
    points = [(r[0],r[1])]

    m = -1 # massa (kg)
    
    N = 500
    dt = 200/N
    e = 0.5
    g = 10

    def f(th, dth, l, dl, t):
        dth2 = (g*np.sin(th)-2*dl*dth)/l
        dl2 = -g*np.cos(th) + l*dth**2
        return np.array([dth, dth2, dl, dl2], float)
        

    for i in range(N):
        for event in pg.event.get():
            if event.type == pg.QUIT:
                return None
            if event.type == pg.VIDEORESIZE:
                disp = pg.display.set_mode((event.w, event.h),pg.RESIZABLE)
                L = np.min([event.h, event.w])
        disp.fill((0,0,0)) # refresh fundo da pagina

        dw.circle(disp, (255,255,255), C, 10, 0)
        
        t = i*dt
        k1 = dt * f(*sol, t)
        k2 = dt * f(*sol + 0.5*k1, t + 0.5*dt)
        k3 = dt * f(*sol + 0.5*k2, t + 0.5*dt)
        k4 = dt * f(*sol + k3, t + dt)

        sol = sol + 1/6*(k1 + 2*k2 + 2*k3 + k4)
        comps.append(sol[2])
        r = C.copy()
        r[0] += sol[2] * np.sin(sol[0])
        r[1] -= sol[2] * np.cos(sol[0])

        points.append((r[0],r[1]))
        if i % 3 == 0: # 0,3,6,9 
            if colorTraj[0] + 2 > 255: colorTraj[0] = 0
            colorTraj[0] += 2
        if i-1 % 3 == 0: # 1,4,7,10
            if colorTraj[1] + 4 > 255: colorTraj[1] = 0
            colorTraj[1] += 4
        if i-2 % 3 == 0: # 2,5,8,11
            if colorTraj[2] - 2 < 0: colorTraj[2] = 255
            colorTraj[2] -= 2
        dw.lines(disp, colorTraj, False, points, 2)

        dw.circle(disp, (255,255,255), r, R, 0)
        dw.line(disp, (255,255,255), C, r)

        pg.display.update()
        clk.tick(FPS)

animate()
pg.quit()

